In [3]:
%load_ext autoreload
%autoreload 2
import torch
from alphatoe import plot, game
from transformer_lens import HookedTransformer, HookedTransformerConfig
import json
import einops
import circuitsvis as cv
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from functools import partial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
weights = torch.load("../scripts/models/prob all 8 layer control-20230718-185339.pt")
with open("../scripts/models/prob all 8 layer control-20230718-185339.json", "r") as f:
    args= json.load(f)

In [5]:
model_cfg = HookedTransformerConfig(
        n_layers=args["n_layers"],
        n_heads=args["n_heads"],
        d_model=args["d_model"],
        d_head=args["d_head"],
        d_mlp=args["d_mlp"],
        act_fn=args["act_fn"],
        normalization_type=args["normalization_type"],
        d_vocab=11,
        d_vocab_out=10,
        n_ctx=10,
        init_weights=True,
        device=args["device"],
        seed=args["seed"])

In [6]:
model =  HookedTransformer(model_cfg)
model.cfg.use_attn_result = True
model.load_state_dict(weights)

<All keys matched successfully>

In [13]:
def ablate_output(module, input, output):
    zeros = [0,2,4,6]
    print(output.shape)
    for i in zeros:
        output[0,:,i] = 0
    return output


In [ ]:
handle.remove()

In [14]:
try:
    handle.remove()
except:
    pass
handle = model.blocks[0].attn.hook_result.register_forward_hook(ablate_output)

In [22]:
seq = [10,1,2,3,4]
with torch.no_grad():
    logits, cache = model.run_with_cache(torch.tensor(seq))

torch.Size([1, 5, 8, 128])


In [16]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (ln2): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)

scores are pre softmax
pattern is post softmax

In [17]:
h_out = cache["result", 0]
h_contrib = h_out @ model.W_U
mlp_out = cache["mlp_out", 0]
mlp_contrib = mlp_out @ model.W_U

In [18]:
# Seq x Head count x Dict size
h_contrib.shape

torch.Size([1, 5, 8, 10])

In [19]:
for i in range(h_contrib.shape[1]):
    plot.imshow(h_contrib[:, i, :])



In [23]:
h_list = [h_contrib[0, -1, i, :] for i in range(h_contrib.shape[2])]
mlp_list = [mlp_contrib[0,-1]]
logit_list =  [logits[0, -1]]

In [24]:
full_list = h_list + mlp_list + logit_list

In [25]:
plot.imshow(torch.stack(full_list), show=False)